In [ ]:
# NLP task - google colab
# huggingface generate function with model llama3.2-3b
# load hugging face model (falcon) in 4 bits (instead of 32 bits) "quantization"
# generate general prompts for the model using generation function
# calculate probability of a sentence given a prompt using the generation function (look into params), calculate the softmax prob of each token in sentence and multiply to get probability of sentence

In [ ]:
# Install huggingface hub
# !pip install --upgrade huggingface_hub

# Get your token from huggingface.co/settings/tokens
from huggingface_hub import login
login()  # This will prompt for your token



In [ ]:
!pip install transformers==4.36 bitsandbytes==0.40.2 accelerate einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 70.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.30.0
    Uninstalling transformers-4.30.0:
      Successfully uninstalled transformers-4.30.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.36.0 which is incompatible.


In [ ]:
#!pip install bitsandbytes accelerate einops

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from torch.nn.functional import softmax
import math

def load_model_and_tokenizer():
    """
    Load the Falcon model and tokenizer with 4-bit quantization
    """
    # Using Falcon-7B-Instruct for better performance on various tasks
    model_name = "tiiuae/falcon-7b-instruct"

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token

    # quantization_config = BitsAndBytesConfig(
    #         load_in_4bit=True,
    #         bnb_4bit_compute_dtype=torch.float16
    #     )

    # Load model in 4-bit precision
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        #quantization_config=quantization_config,
        load_in_4bit=True,
        torch_dtype=torch.float16,
        trust_remote_code=True
    )

    return model, tokenizer


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [ ]:

def generate_text(prompt, model, tokenizer, max_length=100, num_return_sequences=1):
    """
    Generate text using the model
    """
    # Format prompt for Falcon
    formatted_prompt = f"User: {prompt}\nAssistant:"

    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.2
    )

    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]




In [ ]:
model, tokenizer = load_model_and_tokenizer()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_n

model.safetensors.index.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [ ]:
# Example 1: Generate text
prompt = "Write a short story about a robot learning to paint:"
print("\nGenerating story...")
generated_texts = generate_text(prompt, model, tokenizer)
print("\nGenerated Story:")
print(generated_texts[0])


Generating story...

Generated Story:
User: Write a short story about a robot learning to paint:
Assistant: As the sun rose, Robot Paint began to stir. He had been given a brush and some paints, but he was unsure of how to begin. He watched as humans painted intricate designs on their walls, hoping that one day he could learn to do the same. 

Robot Paint practiced every day, dipping his brush in the paint and splashing it onto the canvas. At first, his paintings were messy and inaccurate


In [ ]:
import math
from transformers import LogitsProcessor
from torch.nn.functional import softmax

class ForcedGenerationLogitsProcessor(LogitsProcessor):
    """Custom logits processor that forces the model to generate specific tokens"""
    def __init__(self, target_token_ids):
        self.target_token_ids = target_token_ids
        self.current_pos = 0

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor) -> torch.FloatTensor:
        if self.current_pos < len(self.target_token_ids):
            # Create a mask of -inf for all tokens except the target token
            mask = torch.full_like(scores, float('-inf'))
            mask[:, self.target_token_ids[self.current_pos]] = 0
            scores = scores + mask
            self.current_pos += 1
        return scores

In [ ]:
def calculate_sentence_probability(prompt, sentence, model, tokenizer):
    """
    Calculate the probability of a sentence given a prompt using forced generation

    Args:
        prompt (str): The context prompt
        sentence (str): The sentence to calculate probability for
        model: The language model
        tokenizer: The tokenizer

    Returns:
        float: The probability of the sentence
        list: Individual token probabilities
    """
    # Tokenize the sentence and prompt separately
    sentence_tokens = tokenizer(sentence, return_tensors="pt", add_special_tokens=False)["input_ids"][0]
    prompt_tokens = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)["input_ids"][0]

    # Initialize the logits processor
    logits_processor = ForcedGenerationLogitsProcessor(sentence_tokens.tolist())

    # Generate tokens while forcing the specific sequence
    #input_ids = prompt_tokens.unsqueeze(0).to(model.device)
    input_ids = prompt_tokens.unsqueeze(0)
    token_probs = []
    token_strings = []

    for i in range(len(sentence_tokens)):
        with torch.no_grad():
            outputs = model(input_ids=input_ids)
            next_token_logits = outputs.logits[:, -1, :]

            # Get true probability before masking
            next_token = sentence_tokens[len(token_probs)]
            token_probs_unmasked = softmax(next_token_logits, dim=-1)
            true_token_prob = token_probs_unmasked[0, next_token].item()
            token_probs.append(true_token_prob)

            # Store the actual token string for analysis
            decoded_string = tokenizer.decode(next_token)
            #print("current string: ", decoded_string)
            token_strings.append(decoded_string)
            #print("prob: ", true_token_prob)

            # Process logits to force the next token
            processed_logits = logits_processor(input_ids, next_token_logits)
            next_token_probs = softmax(processed_logits, dim=-1)

            # Print current step
            current_context_text = tokenizer.decode(input_ids[0])
            print(f"\nStep {i+1}:")
            print(f"Current context: '{current_context_text}'")
            print(f"Next token: '{decoded_string}'")
            print(f"P({decoded_string}|{current_context_text}) = {true_token_prob:.4f}")

            # Append the token to input_ids
            input_ids = torch.cat([input_ids, next_token.unsqueeze(0).unsqueeze(0)], dim=1) # current context

    # Calculate total sentence probability
    total_log_prob = sum(math.log(p) for p in token_probs)
    sentence_prob = math.exp(total_log_prob)


    return sentence_prob, token_probs, token_strings

In [ ]:
def analyze_sentence_probability(prompt, sentence, model, tokenizer):
    """
    Analyze the probability of a sentence with detailed token-level breakdown
    """
    sentence_prob, token_probs, token_strings = calculate_sentence_probability(
        prompt, sentence, model, tokenizer
    )

    # print(f"\nAnalyzing probability for sentence: '{sentence}'")
    print("\nToken-level breakdown:")
    for token, prob in zip(token_strings, token_probs):
        print(f"Token: {token:15} Probability: {prob:.4f}")

    return sentence_prob, token_probs, token_strings

In [ ]:
def main():
    try:
        # Example usage
        prompt = "The weather today is"
        sentences = [
            " sunny and warm.",
            " purple and singing."
        ]

        print("\nCalculating probabilities for different sentences...")
        for sentence in sentences:
            prob, token_probs, tokens = analyze_sentence_probability(
                prompt, sentence, model, tokenizer
            )
            print(f"\nTotal probability for '{sentence}': {prob:.2e}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        print("\nTroubleshooting tips:")
        print("1. Check if you have enough GPU memory")
        print("2. Verify that the model and tokenizer are properly loaded")
        print("3. Make sure all required packages are installed")

In [ ]:
main()


Calculating probabilities for different sentences...

Step 1:
Current context: 'The weather today is'
Next token: ' sunny'
P( sunny|The weather today is) = 0.0729

Step 2:
Current context: 'The weather today is sunny'
Next token: ' and'
P( and|The weather today is sunny) = 0.4568

Step 3:
Current context: 'The weather today is sunny and'
Next token: ' warm'
P( warm|The weather today is sunny and) = 0.1561

Step 4:
Current context: 'The weather today is sunny and warm'
Next token: '.'
P(.|The weather today is sunny and warm) = 0.3826

Token-level breakdown:
Token:  sunny          Probability: 0.0729
Token:  and            Probability: 0.4568
Token:  warm           Probability: 0.1561
Token: .               Probability: 0.3826

Total probability for ' sunny and warm.': 1.99e-03

Step 1:
Current context: 'The weather today is'
Next token: ' purple'
P( purple|The weather today is) = 0.0000

Step 2:
Current context: 'The weather today is purple'
Next token: ' and'
P( and|The weather today 

In [ ]:
# Single agent DCOP - design natural language problem and come up with cost table (small language model)

# generate a series(1000+? (start small - 20)) of these prompts with a 'desired' cost table in mind for benchmarking and evaluation
# Figure out the process - maybe use chatgpt/llm to come up with prompts

# first generate few examples by hand , 5-10
# then ask chatgpt to generate similar problems
# create json file of dataset (instruction vs. cost table) and evaluate - can also try to give cost table and ask to predict instruction / context

# we want to generate most likely cost table (python array)
# calc prob of cost table

# example - "agent has 5 actions or meetings - i'm busy at this time x, dont like morning meetings, need meeting at time not busy. Morning meeting should have higher cost... cost = infinity when unavailable..."

# need quantitative evaluation process

# generate without logit processor first (without template, response containing chain of thought("step by step" and other methods) reasoning without logit constraint) as logit processor may hurt result
# then use language model with logit processor after obtaining unstructured final result

# try different models

In [ ]:
######################################################################## OLD

# def calculate_sentence_probability(prompt, sentence, model, tokenizer):
#     """
#     Calculate the probability of a sentence given a prompt
#     """
#     # Combine prompt and sentence
#     full_text = prompt + sentence

#     # Tokenize the full text
#     inputs = tokenizer(full_text, return_tensors="pt").to(model.device)

#     # Get the token IDs for just the sentence
#     sentence_tokens = tokenizer(sentence, return_tensors="pt")["input_ids"][0]
#     prompt_length = len(tokenizer(prompt, return_tensors="pt")["input_ids"][0])

#     # Get model outputs
#     with torch.no_grad():
#         outputs = model(**inputs)
#         logits = outputs.logits

#     # Calculate probability for each token in the sentence
#     total_log_prob = 0

#     for i, token_id in enumerate(sentence_tokens):
#         token_logits = logits[0, prompt_length + i - 1]
#         token_probs = softmax(token_logits, dim=0)
#         token_prob = token_probs[token_id].item()
#         total_log_prob += math.log(token_prob)

#     # Convert log probability to probability
#     sentence_prob = math.exp(total_log_prob)

#     return sentence_prob



# Example usage with error handling
# def main():
#     try:
#         print("Loading model and tokenizer...")
#         model, tokenizer = load_model_and_tokenizer()

#         # Example 1: Generate text
#         prompt = "Write a short story about a robot learning to paint:"
#         print("\nGenerating story...")
#         generated_texts = generate_text(prompt, model, tokenizer)
#         print("\nGenerated Story:")
#         print(generated_texts[0])

#         # Example 2: Calculate sentence probability
#         prompt = "The weather today is"
#         sentence1 = " sunny and warm."
#         sentence2 = " purple and singing."

#         print("\nCalculating sentence probabilities...")
#         prob1 = calculate_sentence_probability(prompt, sentence1, model, tokenizer)
#         prob2 = calculate_sentence_probability(prompt, sentence2, model, tokenizer)

#         print("\nSentence Probabilities:")
#         print(f"'{sentence1}': {prob1:.2e}")
#         print(f"'{sentence2}': {prob2:.2e}")

#     except Exception as e:
#         print(f"An error occurred: {str(e)}")
#         print("\nTroubleshooting tips:")
#         print("1. Check if you have enough GPU memory")
#         print("2. Try restarting the Colab runtime")
#         print("3. Make sure all required packages are installed")

# main()

#############################################################